# ECC Pollard Rho
Instructions for use:
1. Ensure that ECC_utils.py and exampleInputRho.txt are in the same directory as this notebook
2. Click 'Run All' to run the Jupyter notebook.
3. The output is saved to a file named 'OutputBasicRho.txt'.

In [ ]:
from ECC_utils import ECC, Point, read_ECC_instance, write_output
import random

## Parameters
Included are the parameters in the example input and a function to read the parameters from a file.

The parameters are stored as global variables so that they can be accessed by any function without having to pass them as arguments

In [ ]:
# Parameters - Example Input
p = 16001
a = 10
b = 1
P = Point(1654, 7208)
n = 8026
Q = Point(5000, 1283)

E = ECC(p, a, b)

In [ ]:
p, a, b, P, n, Q = read_ECC_instance('../task/exampleInputRho.txt')

E = ECC(p, a, b)

## Basic Pollard Rho method for finding a collision

### Point transformation function

In [ ]:
def inc_mod_p(x, p):
    return (x + 1) % p
def double_mod_p(x, p):
    return (2*x) % p

def f(X, c, d):
    """
    Function for transforming an EC Point (X = c_i * P + d_i * Q) to a new, random-looking EC Point (c_i+1 * P + d_i+1 * Q)

    Parameters:
    X (Point): The point
    c (int): c s.t. X = cP + dQ
    d (int): d s.t. X = cP + dQ
    
    Returns:
    Point: The new point, X_i+1 = X_i + P or 2*X_i or X_i + Q
    int: c_i+1, after the transformation is applied
    int: c_i+1, after the transformation is applied
    """
    partition = X.x % 3
    if partition == 0:
        return E.ECPointAddition(X, P), inc_mod_p(c, n), d
    if partition == 1:
        return E.ECPointDoubling(X), double_mod_p(c, n), double_mod_p(d, n)
    if partition == 2:
        return E.ECPointAddition(X, Q), c, inc_mod_p(d, n)

### Basic Pollard Rho Implementation

In [ ]:
def pollard_rho_ECC(E, P, Q):
    """
    Function for finding a collision

    Parameters:
    E (ECC): The curve to work on
    P (Point): The base point
    Q (Point): The point s.t. Q = lP

    Returns:
    int, int, int, int: c, d, c_, d_ s.t cP + dQ = c_P + d_Q
    """
    c = random.randint(1, p-2)
    d = random.randint(1, p-2)
    
    X = E.ECPointAddition(E.ECPointMult(c, P), E.ECPointMult(d, Q))
    X_, c_, d_ = f(X, c, d)

    while X != X_:
        X, c, d = f(X, c, d)

        X_inter, c_inter, d_inter = f(X_, c_, d_)
        X_, c_, d_ = f(X_inter, c_inter, d_inter)
    
    assert E.ECPointAddition( E.ECPointMult(c, P) , E.ECPointMult(d, Q) ) == E.ECPointAddition( E.ECPointMult(c_, P) , E.ECPointMult(d_, Q) ) == X == X_
    return c, d, c_, d_

c, d, c_, d_ = pollard_rho_ECC(E, P, Q)

# Check for case where c = c_, d=d_
if c == c_ or d == d_:
    print("Collision is identical, re-running Pollard Rho to find a new collision")
    c, d, c_, d_ = pollard_rho_ECC(E, P, Q)

# Output to screen and to file
print(f"c={c}, d={d}, c_={c_}, d_={d_}")
write_output(p, a, b, P, n, Q, c, d, c_, d_, "OutputBasicRho.txt")